In [50]:
import random
import numpy as np
import cv2
import glob
import os

In [78]:
def generate_overlay(background, foreground, x, y):

    background_width = background.shape[1]
    background_height = background.shape[0]

    h, w = foreground.shape[0], foreground.shape[1]

    foreground_image = foreground[..., :3]
    mask = (foreground[..., 3:]!=0)*1

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * foreground_image

    return background

In [83]:
def generate_mask(background, foreground, x, y):

    background_width = background.shape[1]
    background_height = background.shape[0]

    h, w = foreground.shape[0], foreground.shape[1]

    foreground_image = (foreground[..., 3]!=0)*255
    mask = (foreground[..., 3]!=0)*1

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * foreground_image

    return background

In [53]:
def generate_xy(foreground, background):
    bg_w = background.shape[1]
    bg_h = background.shape[0]
    fg_w = foreground.shape[1]
    fg_h = foreground.shape[0]
    x_max = bg_w - fg_w
    y_max = bg_h - fg_h
    x = random.sample(range(0,x_max), 40)
    y = random.sample(range(0,y_max), 40)
    return x, y

In [54]:
def foreground_resize(height, foreground):
    width = int(foreground.shape[1] * height / foreground.shape[0])
    # dsize
    dsize = (width, height)
    # resize image
    output = cv2.resize(foreground, dsize)
    return output

In [87]:
overlay_folder = './fg_bg/'
mask_folder = './fg_bg_mask/'
background_folder = './bg/'
foreground_folder = './fg/'

In [ ]:
os.mkdir(overlay_folder)
os.mkdir(mask_folder)

In [88]:
background_paths = glob.glob(background_folder+'*')
foreground_paths = glob.glob(foreground_folder+'*')


for bg_path in background_paths[:1]: #<-- remove [:1] for creating the entire 400k images
    
    #creating folders corresponding to background
    bg_folder_name = (bg_path.split('/')[-1].split('\\')[-1].split('.')[0])+'/'
    os.mkdir(str(overlay_folder+bg_folder_name))
    os.mkdir(str(mask_folder+bg_folder_name))
    
    #reading background image
    background_image = cv2.imread(bg_path)
    img_counter = 1
    
    #looping over all the foreground images
    for fg_path in foreground_paths:
        
        #reading foreground image
        foreground_image_temp = cv2.imread(fg_path, cv2.IMREAD_UNCHANGED)
        
        #resizing foreground image
        foreground_image = foreground_resize(105, foreground_image_temp.copy())
        
        #generating 40 random positions to place foreground
        x, y =  generate_xy(foreground_image, background_image)
        
        #overlaying 20 foreground
        for i in range(20):
            
            #creating copies to process
            bg_image = background_image.copy()
            fg_image = foreground_image.copy()
            
            #creting black background for mask
            bg_black = np.zeros((224, 224), dtype = "uint8") 
            
            overlaid_image = generate_overlay(bg_image, fg_image, x[i], y[i])
            mask_image = generate_mask(bg_black, fg_image, x[i], y[i])
            
            #saving overlaid and mask images
            cv2.imwrite(overlay_folder+bg_folder_name+"img_{0:04}.jpg".format(img_counter), overlaid_image)
            cv2.imwrite(mask_folder+bg_folder_name+"img_{0:04}.jpg".format(img_counter), mask_image)
            
            #increment the image counter
            img_counter+=1
            
            
        #overlaying 20 flipped foreground
        for i in range(20, 40):
            
            #creating copies to process
            bg_image = background_image.copy()
            fg_image = cv2.flip(foreground_image, 1) #flipping foreground image
            
            #creting black background for mask
            bg_black = np.zeros((224, 224), dtype = "uint8") 
            
            overlaid_image = generate_overlay(bg_image, fg_image, x[i], y[i])
            mask_image = generate_mask(bg_black, fg_image, x[i], y[i])
            
            #saving overlaid and mask images
            cv2.imwrite(overlay_folder+bg_folder_name+"img_{0:04}.jpg".format(img_counter), overlaid_image)
            cv2.imwrite(mask_folder+bg_folder_name+"img_{0:04}.jpg".format(img_counter), mask_image)
            
            #increment the image counter
            img_counter+=1